<a href="https://colab.research.google.com/github/Smart-Pigs/Cow_Food_Consumption/blob/main/Data_Cleaning_6051.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Mount Google Drive 
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
#Change path
import os
path = '/content/gdrive/MyDrive/Cow_eating_label/'
os.chdir(path)

#concatenate for outliers detection 

In [25]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import glob
import numpy as np
import pandas as pd

#remove frist two columns of counts
df = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/uncleaned/cow6051_12am_to_9am_1.csv')
column_names = df.columns.tolist()[2:]
print(column_names)

#prepare dataset
folder = 'Cow6051/Final/uncleaned'
output_df='Cow6051/Final/concatenated'
df = pd.DataFrame()
for file in os.listdir(folder):
  if file.endswith(".csv"):
    file_path = os.path.join(folder, file)
    file_df = pd.read_csv(file_path)
    file_df=file_df[column_names]
    file_df["label"] = file_df["label"].replace("grazing", "eating")
    file_df["label"] = file_df["label"].fillna("not_eating")
    file_df["label"] = file_df["label"].replace("ruminating", "not_eating")
    file_df=file_df[~file_df["label"].str.contains("sleeping")]
    df = pd.concat([df, file_df], ignore_index=True)
    print(f'Add {file}')
output = os.path.join(output_df, '6051_all.csv')
df.to_csv(output, index=False)
    


['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg', 'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar', 'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose', 'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose', 'gyro_z_nose', 'label']
Add cow6051_1pm_to_5pm_1.csv
Add cow6051_5pm_to_9pm_1.csv
Add cow6051_12am_to_9am_1.csv
Add cow6051_9am_to_1pm_1.csv
Add cow6051_1pm_to_5pm_2.csv
Add cow6051_12am_to_9am_2.csv
Add cow6051_9pm_to_12am_2.csv
Add cow6051_5pm_to_9pm_2.csv
Add cow6051_9am_to_1pm_2.csv
Add cow6051_1pm_to_5pm_3.csv
Add cow6051_5pm_to_9pm_3.csv
Add cow6051_9pm_to_12pm_3.csv
Add cow6051_12am_to_9am_3.csv


In [ ]:
# 1.5IQR
def Detect_outlier_IQR(df):
  Q1 = df.quantile(0.25)
  Q3 = df.quantile(0.75)
  IQR = Q3 - Q1
  down_threshold = Q1-1.5*IQR
  up_threshold = Q3+1.5*IQR
  return up_threshold,down_threshold


#label set
labels=set(df['label'])
df=pd.read_csv('Cow6051/Final/concatenated/6051_all.csv')
outliers=0
out_percent=0
for label in labels:
  data_item = df[df['label'] == label]
  print(f"Label: {label}")
  for col in data_item.columns[:-1]:
    col_threshold = Detect_outlier_IQR(data_item[col])
    outliers = data_item[(data_item[col]<col_threshold[1])|(data_item[col]>col_threshold[0])].shape[0]
    print(f"{col}: {col_threshold}")
  print("\n")




Label: eating
acc_x_leg: (2.945263671875, -1.4953613281249998)
acc_y_leg: (-9.64866943359375, -10.122399902343748)
acc_z_leg: (0.6340332031250002, -1.7106933593750004)
gyro_x_leg: (0.8536585569381717, -1.5853658914566042)
gyro_y_leg: (0.7317073345184326, -2.195122003555298)
gyro_z_leg: (1.8445121943950653, -2.911585360765457)
acc_x_maxilar: (13.260864257812496, 2.1688720703125037)
acc_y_maxilar: (12.988708496093754, -3.1611938476562518)
acc_z_maxilar: (1.1915039062500008, -7.134667968750001)
gyro_x_maxilar: (75.62500381469727, -75.96036911010742)
gyro_y_maxilar: (52.43902540206909, -51.70731782913208)
gyro_z_maxilar: (61.85975646972656, -64.96951293945312)
acc_x_nose: (3.4536865234375003, -5.4179931640625005)
acc_y_nose: (-3.5368286132812514, -13.504309082031249)
acc_z_nose: (13.271032714843749, -8.056408691406249)
gyro_x_nose: (62.53048801422119, -60.88414669036865)
gyro_y_nose: (59.07012462615967, -60.07622241973877)
gyro_z_nose: (46.875001430511475, -45.68597650527954)


Label: not_

In [ ]:
out_percent=outliers/df.shape[0] 
print(out_percent) 

0.05834787410367136


#Back to original files and remove outliers

In [ ]:
#Label: eating
thres_eat={
  'acc_x_leg': (2.945263671875, -1.4953613281249998),
  'acc_y_leg': (-9.64866943359375, -10.122399902343748),
  'acc_z_leg': (0.6340332031250002, -1.7106933593750004),
  'gyro_x_leg': (0.8536585569381717, -1.5853658914566042),
  'gyro_y_leg': (0.7317073345184326, -2.195122003555298),
  'gyro_z_leg': (1.8445121943950653, -2.911585360765457),
  'acc_x_maxilar': (13.260864257812496, 2.1688720703125037),
  'acc_y_maxilar': (12.988708496093754, -3.1611938476562518),
  'acc_z_maxilar': (1.1915039062500008, -7.134667968750001),
  'gyro_x_maxilar': (75.62500381469727, -75.96036911010742),
  'gyro_y_maxilar': (52.43902540206909, -51.70731782913208),
  'gyro_z_maxilar': (61.85975646972656, -64.96951293945312),
  'acc_x_nose': (3.4536865234375003, -5.4179931640625005),
  'acc_y_nose': (-3.5368286132812514, -13.504309082031249),
  'acc_z_nose': (13.271032714843749, -8.056408691406249),
  'gyro_x_nose': (62.53048801422119, -60.88414669036865),
  'gyro_y_nose': (59.07012462615967, -60.07622241973877),
  'gyro_z_nose': (46.875001430511475, -45.68597650527954),
}


#Label: not_eating
thres_neat = {
  'acc_x_leg': (3.26287841796875, -2.21612548828125),
  'acc_y_leg': (-9.586462402343749, -10.146325683593751),
  'acc_z_leg': (0.7632324218750002, -1.8398925781250002),
  'gyro_x_leg': (0.716463387012482, -1.356707274913788),
  'gyro_y_leg': (0.5945121645927429, -1.9664633870124817),
  'gyro_z_leg': (1.2042683213949206, -2.0884146839380264),
  'acc_x_maxilar': (11.824719238281252, -0.832019042968752),
  'acc_y_maxilar': (12.747058105468753, 2.2627807617187483),
  'acc_z_maxilar': (0.9414794921875003, -7.2794189453125),
  'gyro_x_maxilar': (59.10061025619507, -61.99695062637329),
  'gyro_y_maxilar': (35.92987871170044, -35.41158628463745),
  'gyro_z_maxilar': (34.58841562271118, -35.777440547943115),
  'acc_x_nose': (2.9847412109375004, -4.0207275390625),
  'acc_y_nose': (-4.806689453125003, -12.156689453124997),
  'acc_z_nose': (11.022009277343752, -4.49625244140625),
  'gyro_x_nose': (36.61585283279419, -37.04268217086792),
  'gyro_y_nose': (45.53353786468506, -46.53963565826416),
  'gyro_z_nose': (31.280489206314087, -30.426830053329468),
}




In [ ]:
import glob
import numpy as np
import pandas as pd

Input_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/uncleaned'
Output_path = '/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/'

df = pd.DataFrame()
for file_name in os.listdir(Input_path):
  if file_name.endswith('.csv'):
    file_path = os.path.join(Input_path, file_name)
    df = pd.read_csv(file_path)
    df = df[df.columns.tolist()[2:]]
    df["label"] = df["label"].replace("grazing", "eating")
    df["label"] = df["label"].fillna("not_eating")
    df["label"] = df["label"].replace("ruminating", "not_eating")
    df=df[~df["label"].str.contains("sleeping")]
    for label, threshold in [('eating', thres_eat), ('not_eating', thres_neat)]:
      mask = df['label'] == label
      for col in df.columns[:-1]: 
        if col in threshold:
          # Remove outliers
          upper_thresh, lower_thresh = threshold[col]
          outside_thresh_mask = (df[col] < lower_thresh) | (df[col] > upper_thresh)
          df.loc[outside_thresh_mask & mask, col] = np.nan
          # Interpolate  
          df.loc[mask, col] = df.loc[mask, col].interpolate(method='linear', limit_direction='both')
          # df_new = df_new.append(df)
    clean_file_path = os.path.join(Output_path, f'{file_name}')
    df.to_csv(clean_file_path, index=False)
    print(f'Saved {clean_file_path}')
    df = pd.DataFrame()









Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5pm_to_9pm_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_12am_to_9am_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9am_to_1pm_1.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_12am_to_9am_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9pm_to_12am_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5pm_to_9pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9am_to_1pm_2.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_3.csv
Saved /content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5p

# Windowing+ Feature extraction for each blocks

In [ ]:
import glob
import numpy as np
import pandas as pd

df1 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_1.csv')
df2 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5pm_to_9pm_1.csv')
df3 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_12am_to_9am_1.csv')
df4 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9am_to_1pm_1.csv')
df5 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_2.csv')
df6 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_12am_to_9am_2.csv')
df7 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9pm_to_12am_2.csv')
df8 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5pm_to_9pm_2.csv')
df9 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9am_to_1pm_2.csv')
df10 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_1pm_to_5pm_3.csv')
df11 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_5pm_to_9pm_3.csv')
df12 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_9pm_to_12pm_3.csv')
df13 = pd.read_csv('/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/clean/cow6051_12am_to_9am_3.csv')

In [ ]:
import scipy.stats as stats 
def window_frames(data_list, window_size, step):
    data_len = len(data_list)
    cut_num = int((data_len - window_size) / step)
    window_DataFrames = []
    labels = []
    for i in range(cut_num+1):
        #--------------get data inside window
        start_index = i  * step
        end_index = start_index + window_size
        cur_data = data_list[start_index:end_index]
        cur_data_x=cur_data[['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']]
        cur_data_y=cur_data[['label']]
        #print(cur_data)
        window_DataFrames.append(cur_data_x)
        #--------------get most common label
        labels.append(stats.mode(cur_data_y)[0][0][0])
        #print("window_DataFrames",window_DataFrames)
        #print("labels",labels)
    #window_DataFrames = np.asarray(res)
    #labels = np.asarray(labels)
        
    return window_DataFrames,labels

In [ ]:
def get_timeDomain_features(inputSequence):
    return np.mean(inputSequence),np.std(inputSequence),np.max(inputSequence),np.min(inputSequence),np.median(inputSequence),np.percentile(inputSequence, 75, interpolation='midpoint')-np.percentile(inputSequence, 25, interpolation='midpoint')


In [ ]:
from numpy.fft import fft, fftfreq, ifft 
# Todo: Could add A: mean(A), skewness, kurtosis; Shape: std, skewness, kurtosis
def get_frequency_features(inputSequence,totalwindow_size):
    # real sequence sample data
    m=fftfreq(totalwindow_size)>0 
    # Theoritical fft
    fft_th=np.abs(fft(inputSequence)/totalwindow_size)*2.0
    # Maximum Amplitude Frequency: x_max_f = f(argmax(A))
    max_fre=fftfreq(totalwindow_size)[m][ np.argmax(fft_th[m])]
    # Power : P = 1/(windowsize+1)*|(a)|^(2) = P/sum(P)
    t1=np.square(fft_th[m])
    t2=float(len(fft_th[m]))
    power_spectral_d=np.divide(t1,t2)
    if np.sum(power_spectral_d)==0:
        power_spectral_d_p=np.divide(power_spectral_d,1)*0
    else:
        power_spectral_d_p=np.divide(power_spectral_d,np.sum(power_spectral_d))
    # Power spectral entropy: x_pse = -sum(P*ln(P))
    power_spectral_entr=0 
    if np.count_nonzero(power_spectral_d_p)==power_spectral_d_p.size:
         power_spectral_entr=-np.sum(np.log(power_spectral_d_p)*power_spectral_d_p)
    else:
        power_spectral_entr=0   
    # Weighted Frequencies: weight sum(A*f)/sum(A)
    fre_weighted=0.0
    if np.sum(fft_th[m])==0:
        fre_weighted=(float(np.sum(fftfreq(totalwindow_size)[m]*fft_th[m]))/1)*0
    else:
        fre_weighted=float(np.sum(fftfreq(totalwindow_size)[m] * fft_th[m])) / np.sum(fft_th[m])
    return max_fre,fre_weighted,power_spectral_entr

In [ ]:
def get_features_with_sildingWindow(df, window_size, stride_size):
    
    x, y = window_frames(df, window_size, stride_size)
    new_x = [] 

    for i in range(0, len(x)):  
        batch_features = []
        for label in ['acc_x_leg', 'acc_y_leg', 'acc_z_leg', 'gyro_x_leg', 'gyro_y_leg',
       'gyro_z_leg', 'acc_x_maxilar', 'acc_y_maxilar', 'acc_z_maxilar',
       'gyro_x_maxilar', 'gyro_y_maxilar', 'gyro_z_maxilar', 'acc_x_nose',
       'acc_y_nose', 'acc_z_nose', 'gyro_x_nose', 'gyro_y_nose',
       'gyro_z_nose']:
            #print("x[i]",label,x[i][label])
            mean, std, maxv, minv, median, IQR=get_timeDomain_features(x[i][label])
            #print("mean, std, maxv, minv, median, IQR",mean, std, maxv, minv, median, IQR)
            max_fre,fre_weighted,power_spectral_entr=get_frequency_features(x[i][label],window_size)
            #print("max_fre,fre_weighted,power_spectral_entr",max_fre,fre_weighted,power_spectral_entr)
            batch_features.append(
                [ mean, std, maxv, minv, median, IQR,max_fre,fre_weighted,power_spectral_entr])
#             if i>2:
#                 break;
        
        batch_features = np.array(batch_features)  # Convert to numpy so we can reshape 
        #print("batch_features.shape",batch_features.shape)
        new_x.append(batch_features.reshape((1,
                                         batch_features.shape[0]*batch_features.shape[1])).tolist()[0])
    #print(new_x)
        
       

    return new_x, y

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_names = [df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13]
df_names_str=['cow6051_1pm_to_5pm_1','cow6051_5pm_to_9pm_1','cow6051_12am_to_9am_1','cow6051_9am_to_1pm_1','cow6051_1pm_to_5pm_2','cow6051_12am_to_9am_2','cow6051_9pm_to_12am_2','cow6051_5pm_to_9pm_2','cow6051_9am_to_1pm_2','cow6051_1pm_to_5pm_3','cow6051_5pm_to_9pm_3','cow6051_9pm_to_12pm_3','cow6051_12am_to_9am_3']


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
windowsize = 128
stride_size = 128
np.set_printoptions(suppress=True)
count=-1


for df in df_names:
  df_raw = df
  df_raw_np = df_raw.to_numpy()
  X = df_raw_np[:,:-1]
  y = df_raw_np[:,-1]
  print(X.shape, y.shape)
  raw = np.concatenate((X,y.reshape(len(y),1)), axis=1)
  raw_df = pd.DataFrame(raw, columns=df_raw.columns)
  df_win_X, y = get_features_with_sildingWindow(raw_df, window_size=windowsize, stride_size=stride_size)
  print(len(df_win_X),len(y))

  df_win_X_np = np.array(df_win_X)
  print(df_win_X_np.shape)

  scaler = StandardScaler().fit(df_win_X_np)
  df_win_X_np = scaler.transform(df_win_X_np)
  y_np = np.array(y,dtype=str)
  count += 1

  np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/Window_1/"+df_names_str[count]+"_x.csv", df_win_X_np, delimiter=",")
  np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6051/Final/Window_1/"+df_names_str[count]+"_y.csv", y_np, delimiter=",",fmt='%s')
  print(f'saved {df_names_str[count]}')







(266193, 18) (266193,)
2079 2079
(2079, 162)
saved cow6051_1pm_to_5pm_1
(133114, 18) (133114,)
1039 1039
(1039, 162)
saved cow6051_5pm_to_9pm_1
(133139, 18) (133139,)
1040 1040
(1040, 162)
saved cow6051_12am_to_9am_1
(308669, 18) (308669,)
2411 2411
(2411, 162)
saved cow6051_9am_to_1pm_1
(310555, 18) (310555,)
2426 2426
(2426, 162)
saved cow6051_1pm_to_5pm_2
(154764, 18) (154764,)
1209 1209
(1209, 162)
saved cow6051_12am_to_9am_2
(88748, 18) (88748,)
693 693
(693, 162)
saved cow6051_9pm_to_12am_2
(150005, 18) (150005,)
1171 1171
(1171, 162)
saved cow6051_5pm_to_9pm_2
(231318, 18) (231318,)
1807 1807
(1807, 162)
saved cow6051_9am_to_1pm_2
(339430, 18) (339430,)
2651 2651
(2651, 162)
saved cow6051_1pm_to_5pm_3
(131955, 18) (131955,)
1030 1030
(1030, 162)
saved cow6051_5pm_to_9pm_3
(16784, 18) (16784,)
131 131
(131, 162)
saved cow6051_9pm_to_12pm_3
(22573, 18) (22573,)
176 176
(176, 162)
saved cow6051_12am_to_9am_3


## Test cow6065_1pm_to_5pm_2


In [29]:
dftest = pd.concat([df1,df2], ignore_index=True)
dftrain = pd.concat([df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13], ignore_index=True)

dftest_np =dftest.to_numpy()
dftrain_np =dftrain.to_numpy()

X_train = dftrain_np[:,:-1]
X_test = dftest_np[:,:-1]

y_train = dftrain_np[:,-1]
y_test = dftest_np[:,-1]

In [30]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)
     

(1887940, 18)
(399307, 18)
(1887940,)
(399307,)


In [31]:
train = np.concatenate((X_train,y_train.reshape(len(y_train),1)), axis=1)
test = np.concatenate((X_test,y_test.reshape(len(y_test),1)), axis=1)
print(train.shape)
print(test.shape)

(1887940, 19)
(399307, 19)


In [32]:
df_train = pd.DataFrame(train, columns=dftrain.columns)
df_test = pd.DataFrame(test, columns=dftest.columns)

In [33]:
windowsize = 128
stride_size = 128
df_ml_train, y_train = get_features_with_sildingWindow(df_train, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_train),len(y_train))

df_ml_test, y_test = get_features_with_sildingWindow(df_test, window_size=windowsize, stride_size=stride_size)
print(len(df_ml_test),len(y_test))

14749 14749
3119 3119


In [34]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_train)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_train, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()


In [35]:
from sklearn.decomposition import PCA
import plotly.express as px
pca = PCA(2)  # project from 64 to 2 dimensions
projected = pca.fit_transform(df_ml_test)
#print(df_ml.shape)
#print(projected.shape)
#print(projected)
#y_string = y.astype(str)
fig = px.scatter(projected, x=1, y=0, color = y_test, color_discrete_map={0: 'blue', 1: 'blue', 2 : 'darkblue', 3:'blue'})
fig.show()

In [36]:
df_np_ml_train = np.array(df_ml_train)
df_np_ml_test = np.array(df_ml_test)
print(df_np_ml_train.shape)
print(df_np_ml_test.shape)

(14749, 162)
(3119, 162)


In [37]:
np.set_printoptions(suppress=True)
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df_np_ml_train)
df_np_ml_train = scaler.transform(df_np_ml_train)
df_np_ml_test = scaler.transform(df_np_ml_test)
y_np_train = np.array(y_train,dtype=str)
y_np_test = np.array(y_test,dtype=str)

In [38]:
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_train_df1_df2.csv", df_np_ml_train, delimiter=",")
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/X_test_df1_df2.csv", df_np_ml_test, delimiter=",")
   
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_df1_df2.csv", y_np_train, delimiter=",",fmt='%s')
np.savetxt("/content/gdrive/MyDrive/Cow_eating_label/Cow6065/Final/featured/y_train_df1_df2.csv", y_np_test, delimiter=",",fmt='%s')
     

In [39]:
from sklearn.ensemble import RandomForestClassifier
rfc_clf = RandomForestClassifier()
rfc_clf.fit(df_np_ml_train, y_np_train)
yhat = rfc_clf.predict(df_np_ml_test)

from sklearn.metrics import classification_report
print(classification_report(y_np_test, yhat))

              precision    recall  f1-score   support

      eating       0.98      1.00      0.99      1806
  not_eating       1.00      0.97      0.98      1313

    accuracy                           0.99      3119
   macro avg       0.99      0.98      0.99      3119
weighted avg       0.99      0.99      0.99      3119

